# 1.문제 
- 출발 공항에 대해서 도착 공항별로 평균 출발 지연시간 평균 도착지연시간을 구해서 DataFrame을 생성


In [ ]:
#heidi 도구 csv가져오기, 제어문자는 , 테이블은 새 테이블로하고 중복행처리는 insert

In [ ]:
# pre training or 전이학습

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
hflight = pd.read_csv('./hflight.csv')

In [9]:
hflight_group = hflight.groupby(["Origin","Dest"])[["ArrDelay","DepDelay"]].mean().reset_index()

In [3]:
hflight_group = hflight.groupby(["Origin","Dest"],as_index=False)[["ArrDelay","DepDelay"]].mean()

In [ ]:
# CREATE TABLE hflight_group
# SELECT Origin, Dest,AVG(ArrDelay),AVG(DepDelay)
# FROM hflight
# GROUP BY Origin,Dest	

In [ ]:
# SELECT Origin, Dest, round(AVG(ArrDelay),2) Arrive_Delay,  round(AVG(DepDelay),2) DepDelay
# FROM hflight
# GROUP BY Origin,Dest	

# 문제 2
- 목적지 공항에 대해 연착 건수를 구하고, 연착 건수가 2000회 이상인 공항에 대한 데이터만 추출
- col -> Dest :목적지 공항 ArrDelay (연착은 5분이상)

In [9]:
hflight.isnull().sum()

Year                      0
Month                     0
DayofMonth                0
DayOfWeek                 0
DepTime                2905
ArrTime                3066
UniqueCarrier             0
FlightNum                 0
TailNum                 795
ActualElapsedTime      3622
AirTime                3622
ArrDelay               3622
DepDelay               2905
Origin                    0
Dest                      0
Distance                  0
TaxiIn                 3066
TaxiOut                2947
Cancelled                 0
CancellationCode     224523
Diverted                  0
dtype: int64

In [13]:
tmp2 = hflight[hflight.ArrDelay >=5].groupby(["Dest"],as_index=False)[["Year"]].count() 

In [17]:
hflight2 = hflight.Dest.isin(tmp2[tmp2.Year>=2000].Dest)

In [34]:
hflight2.describe()

count     227496
unique         2
top        False
freq      185235
Name: Dest, dtype: object

In [20]:
hflight_2_result = hflight[hflight2].copy()

In [21]:
hflight_2_result.Dest.value_counts()

Dest
DAL    9820
ATL    7886
MSY    6823
LAX    6064
DEN    5920
ORD    5748
Name: count, dtype: int64

In [14]:
hflight["ArrDelay_is"] = hflight["ArrDelay"].apply(lambda x : 1 if x>=5  else 0)

In [19]:
hflight_delay_group = hflight.groupby(["Dest"])["ArrDelay_is"].sum().reset_index()

In [21]:
hflight_delay_group_2000 = hflight_delay_group[hflight_delay_group.ArrDelay_is>=2000]

In [88]:
hflight_delay_group_2000

,Dest,ArrDelay_is
6,ATL,2657
28,DAL,3360
31,DEN,2315
59,LAX,2641
78,MSY,2361
84,ORD,2220


In [ ]:
# SELECT B.Dest, B.ad
# FROM
# (SELECT A.DEST, SUM(A.ArrDelay_is) ad
# From
# (SELECT Dest,CASE WHEN ArrDelay >=5 THEN 1
# ELSE 0 END ArrDelay_is	
# FROM hflight) A
# GROUP BY A.DEST) B
# WHERE B.ad>=2000

In [ ]:
# SELECT Dest, COUNT(DAYOFWEEK) cnt
# FROM hflight
# WHERE ArrDelay >=5
# GROUP BY Dest
# HAVING cnt>=2000

# 문제 3
-  위의 결과를 바탕으로 목적지 공항 별 결항 횟수, 회항 횟수
- 운항 횟수를 구하시오 (Cancelled, Diverted, Air)
- 운항 횟수는 결항과 회항을 제외한 것 

In [24]:
a = hflight_2_result.groupby(["Dest"])["Cancelled"].sum()

In [25]:
b = hflight_2_result.groupby(["Dest"])["Diverted"].sum()

In [26]:
c = hflight_2_result.groupby(["Dest"])["Year"].count()

In [29]:
d  = c-b-a

In [32]:
# pd.concat([a,b,d],axis=1).rename(columns={0 : "air"})
pd.concat([a,b,(c-b-a)],axis=1).rename(columns={0 : "air"})

,Cancelled,Diverted,air
Dest,,,
ATL,141,28,7717
DAL,442,27,9351
DEN,28,24,5868
LAX,33,14,6017
MSY,40,3,6780
ORD,99,15,5634


In [28]:
hflight_count_group = hflight.groupby(["Dest"])[["Cancelled","Diverted"]].sum().reset_index()

In [46]:
hflight_count_group_only = hflight.groupby(["Dest"])["ArrDelay_is"].count().reset_index()

In [48]:
hflight_count_group = pd.merge(hflight_count_group,hflight_count_group_only, on="Dest", how="inner")

In [50]:
hflight_count_group["real"] = hflight_count_group["ArrDelay_is"] - hflight_count_group["Cancelled"] - hflight_count_group["Diverted"]

In [53]:
hflight_count_group_ans = hflight_count_group[["Dest","real"]]

In [55]:
# hflight_count_group_ans

In [89]:
hflight.iloc[0]

Year                   2011
Month                     1
DayofMonth                1
DayOfWeek                 6
DepTime              1400.0
ArrTime              1500.0
UniqueCarrier            AA
FlightNum               428
TailNum              N576AA
ActualElapsedTime      60.0
AirTime                40.0
ArrDelay              -10.0
DepDelay                0.0
Origin                  IAH
Dest                    DFW
Distance                224
TaxiIn                  7.0
TaxiOut                13.0
Cancelled                 0
CancellationCode        NaN
Diverted                  0
ArrDelay_is               0
Name: 0, dtype: object

In [ ]:
# sudo apt install mysql-server
# sudo service mysql start
# 해보고
# sudo service mysql status

In [69]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, BigInteger, DateTime, Date,Float

In [70]:
import sqlalchemy
from urllib import parse
user = 'id'
password = 'pw'
host='ip'
port = port
database = 'kpi'
password = parse.quote_plus(password)
engine = sqlalchemy.create_engine(f"mysql://{user}:{password}@{host}:{port}/{database}")

In [71]:
metadata = MetaData()

In [81]:
# hflight.columns

In [82]:
# hflight.iloc[0]

In [83]:
# hflight.info()

In [78]:
Table("hflight", metadata, 
      Column('Year', Integer),
      Column("Month", Integer),
      Column("DayofMonth", Integer),
      Column("DayOfWeek", Integer),
      Column("DepTime", Float),
      Column("ArrTime", Float),
      Column("UniqueCarrier", String(100)),
      Column("FlightNum", Integer),
      Column("TailNum", String(100)),
      Column("ActualElapsedTime", Float),
      Column("AirTime", Float),
      Column("ArrDelay", Float),
      Column("DepDelay", Float),
      Column("Origin", String(100)),
      Column("Dest", String(100)),
      Column("Distance", Integer),
      Column("TaxiIn", Float),
      Column("TaxiOut", Float),
      Column("Cancelled", Integer),
      Column("CancellationCode", String(100)),
      Column("Diverted", Integer),
      Column("ArrDelay_is", Integer)
     )

Table('hflight', MetaData(), Column('Year', Integer(), table=<hflight>), Column('Month', Integer(), table=<hflight>), Column('DayofMonth', Integer(), table=<hflight>), Column('DayOfWeek', Integer(), table=<hflight>), Column('DepTime', Float(), table=<hflight>), Column('ArrTime', Float(), table=<hflight>), Column('UniqueCarrier', String(length=100), table=<hflight>), Column('FlightNum', Integer(), table=<hflight>), Column('TailNum', String(length=100), table=<hflight>), Column('ActualElapsedTime', Float(), table=<hflight>), Column('AirTime', Float(), table=<hflight>), Column('ArrDelay', Float(), table=<hflight>), Column('DepDelay', Float(), table=<hflight>), Column('Origin', String(length=100), table=<hflight>), Column('Dest', String(length=100), table=<hflight>), Column('Distance', Integer(), table=<hflight>), Column('TaxiIn', Float(), table=<hflight>), Column('TaxiOut', Float(), table=<hflight>), Column('Cancelled', Integer(), table=<hflight>), Column('CancellationCode', String(length

In [79]:
metadata.create_all(engine)
customers.to_sql("hflight", if_exists='append', con=engine, index=False)
# customers.to_sql("hflight", if_exists='replace', con=engine, index=False) #이걸로하면 table 만들지않고서도 넣을수있다.

227496

In [ ]:
# SELECT A.Dest , A.a3-A.a2-A.a1
# FROM 
# (SELECT Dest, SUM(Cancelled) a1, SUM(Diverted) a2, COUNT(Cancelled) a3
# FROM hflight
# GROUP BY Dest) A


In [ ]:
# CREATE VIEW hflight2 AS
# SELECT * FROM hflight WHERE Dest IN
# 	(SELECT Dest FROM 
# 		(SELECT Dest, COUNT(DAYOFWEEK) cnt
# 		FROM hflight
# 		WHERE ArrDelay >=5
# 		GROUP BY Dest
# 		HAVING cnt>=2000
# 		) 
# 	a) 

# SELECT Dest, COUNT(*) total 
# FROM hflight2 
# GROUP BY Dest;

	
	
# SELECT Dest, COUNT(*) total
# FROM hflight2
# WHERE Cancelled =1 
# GROUP BY Dest 

# SELECT Dest, COUNT(*) total
# FROM hflight2
# WHERE Diverted =1 
# GROUP BY Dest 


# SELECT a.Dest, a.total, b.Cancelled, c.Diverted, ( a.total- b.Cancelled- c.Diverted) AS air
# FROM (SELECT Dest, COUNT(*) total
# 		FROM hflight2
# 		GROUP BY Dest ) a
# LEFT JOIN(
# 		SELECT Dest, COUNT(*) Cancelled
# 		FROM hflight2
# 		WHERE Cancelled =1 
# 		GROUP BY Dest 
# 	) b
# ON a.Dest = b.Dest
# LEFT JOIN (
# 		SELECT Dest, COUNT(*) Diverted
# 		FROM hflight2
# 		WHERE Diverted =1 
# 		GROUP BY Dest 	
# 		) c 
#  ON a.Dest=c.Dest